"![FMD_Overview](https://github.com/edkreuk/FMD_FRAMEWORK/blob/main/Images/FMD_Overview.png?raw=true)"

In [ ]:
%pip install ms-fabric-cli --quiet

## Create Tokens

In [ ]:
# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token

tasks=[]


In [ ]:
import subprocess
import os
from time import sleep, time
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import sempy.fabric as fabric
import struct
import pyodbc

## Configuration and Parameters

In [ ]:
load_demo_data_data= False         # Set to True if you want to load the demo data, otherwise set to False
lakehouse_schema_enabled = True    # Set to True if you want to use the lakehouse schema, otherwise set to False

driver = '{ODBC Driver 18 for SQL Server}' # Change this if you use a different driver
# target connections guid 
# add the correct id for every connections
# change the capacity id
# change the workspace roles

FrameworkName= 'FMD' # max 6 characters for better visibility, no spaces and the end of the name

capacity_name_dvlm = '***********************'                  # Which capacity will be used for these workspaces in development
capacity_name_prod = '***********************'                  # Which capacity will be used for these workspaces in production
capacity_name_config = '***********************'                #'Which capacity will be used for this workspace for the FMD Database


## original data

workspace_roles = [ # Keep emtpy [] if you only want to assign this to your personal account
                    {
                        "principal": {
                            "id": '***********************',
                            "displayName": "sg-fmd-fabric-contributor", # Name of the group or user to assign the role to
                            "type": "Group"
                        },
                        "role": "contributor"  #(choose from 'admin', 'member', 'contributor', 'viewer')

                        }
                    ]


configuration = {
                    'workspace': {
                        'name' : FrameworkName + ' CONFIG FMD',             # Name of target workspace
                        'roles' : workspace_roles,                          # Roles to assign to the workspace
                        'capacity_name' : capacity_name_config              # Name of target capacity for the configuration workspace
                    },
                       'DatabaseName' : 'SQL_'+FrameworkName+'_FRAMEWORK'   # Name of target configuration SQL Database
}
environments = [
                    {
                        'environment_name' : 'development',                 # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (D) FMD',   # Name of target code workspace for development
                                'roles' : workspace_roles,                  # Roles to assign to the workspace
                                'capacity_name' : capacity_name_dvlm        # Name of target data workspace for development
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (D) FMD',   # Name of target data workspace for development
                                'roles' : workspace_roles,                  # Roles to assign to the workspace
                                'capacity_name' : capacity_name_dvlm        # Name of target code workspace for development
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',          # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',    # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ADF_PIPELINES' : None                                          #'10000000-0000-0000-0000-000000000000'

                        }
                    },
                    {
                        'environment_name' : 'production',                  # Name of target environment
                        'workspaces': {
                            'data' : {
                                'name' : FrameworkName + ' DATA (P) FMD',   # Name of target data workspace for production
                                'roles' : workspace_roles,                  # Roles to assign to the workspace
                                'capacity_name' : capacity_name_prod        # Name of target data workspace for production   
                            },
                            'code' : {
                                'name' : FrameworkName + ' CODE (P) FMD',   # Name of target code workspace for production
                                'roles' : workspace_roles,                  # Roles to assign to the workspace
                                'capacity_name' : capacity_name_prod        # Name of target code workspace for production
                            }
                        },
                        'connections' : {
                            'CON_FMD_FABRIC_SQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',          # Required Guid to the Fabric SQL connection
                            'CON_FMD_FABRIC_PIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',    # Required Guid to the Fabric datapipelines connection
                            'CON_FMD_ADF_PIPELINES' : None                                          #'10000000-0000-0000-0000-000000000000'

                        }
                    }
                ]



## Repo Configuration

In [ ]:
#FMD Framework code
##### DO NET CHANGE UNLESS SPECIFIED OTHERWISE ####
repo_owner = "edkreuk"              # Owner of the repository
repo_name = "FMD_FRAMEWORK"         # Name of the repository
branch = "main"                     #"main" is default                    
folder_prefix = ""
###################################################
     

## Download source & config files

In [ ]:
def download_folder_as_zip(repo_owner, repo_name, output_zip, branch="main", folder_to_extract="src",  remove_folder_prefix = ""):
    # Construct the URL for the GitHub API to download the repository as a zip file
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"
    
    # Make a request to the GitHub API
    response = requests.get(url)
    response.raise_for_status()

    folder_to_extract = f"/{folder_to_extract}" if folder_to_extract[0] != "/" else folder_to_extract
    
    # Ensure the directory for the output zip file exists
    os.makedirs(os.path.dirname(output_zip), exist_ok=True)
    
    # Create a zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zipf:
        with zipfile.ZipFile(output_zip, 'w') as output_zipf:
            for file_info in zipf.infolist():
                parts = file_info.filename.split('/')
                if  re.sub(r'^.*?/', '/', file_info.filename).startswith(folder_to_extract): 
                    # Extract only the specified folder
                    file_data = zipf.read(file_info.filename)  
                    if folder_prefix != "":
                        parts.remove(remove_folder_prefix)
                    output_zipf.writestr(('/'.join(parts[1:])), file_data)

def uncompress_zip_to_folder(zip_path, extract_to):
    # Ensure the directory for extraction exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Uncompress all files from the zip into the specified folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Delete the original zip file
    os.remove(zip_path)

download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/src/src.zip", branch = branch, folder_to_extract= f"{folder_prefix}/src", remove_folder_prefix = f"{folder_prefix}")
download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/config/config.zip", branch = branch, folder_to_extract= f"{folder_prefix}/config" , remove_folder_prefix = f"{folder_prefix}")
uncompress_zip_to_folder(zip_path = "./builtin/config/config.zip", extract_to= "./builtin")
 

## CLI Login

In [ ]:
# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token

tasks=[]


## Get connections

In [ ]:
result = subprocess.run(["fab", "api", "-X", "get", "connections"], capture_output=True, text=True)

#print("STDOUT:", result.stdout)
#print("STDERR:", result.stderr)
connections=json.loads(result.stdout)["text"]
connection_list = [item['id'] for item in connections['value']]



## Deployment functions

In [ ]:
# -------------------------------
# FABRIC CLI Utilities
# -------------------------------

def run_fab_command(command, capture_output=False, silently_continue=False, raw_output=False):
    """
    Executes a Fabric CLI command with optional output capture and error handling.
    """
    result = subprocess.run(["fab", "-c", command], capture_output=capture_output, text=True)
    if not silently_continue and (result.returncode > 0 or result.stderr):
        raise Exception(f"Error running fab command. exit_code: '{result.returncode}'; stderr: '{result}'")
    if capture_output:
        return result if raw_output else result.stdout.strip()
    return None

# -------------------------------
# Workspace Management
# -------------------------------

def get_workspace_id_by_name(workspace_name):
    """
    Retrieves the workspace ID by its display name.
    """
    result = run_fab_command("api -X get workspaces/", capture_output=True, silently_continue=True)
    workspaces = json.loads(result)["text"]["value"]
    normalized_name = workspace_name.strip().lower()
    match = next((w for w in workspaces if w['displayName'].strip().lower() == normalized_name), None)
    return match['id'] if match else None

def ensure_workspace_exists(workspace):
    """
    Ensures the workspace exists; creates it if not found.
    """
    workspace_name = workspace['name']
    workspace_id = get_workspace_id_by_name(workspace_name)
    if workspace_id:
        print(f" - Workspace '{workspace_name}' found. Workspace ID: {workspace_id}- assign capacity: {workspace['capacity_name']}")
        run_fab_command(f'assign ".capacities/{workspace["capacity_name"]}.Capacity" -W "{workspace_name}.Workspace" -f', silently_continue=True)
        return workspace_id, "exists"

    print(f" - Workspace '{workspace_name}' not found. Creating new workspace...")
    run_fab_command(f'mkdir "{workspace_name}.workspace" -P capacityName="{workspace["capacity_name"]}"', silently_continue=True)
    workspace_id = get_workspace_id_by_name(workspace_name)
    if workspace_id:
        print(f" - Created workspace '{workspace_name}'. ID: {workspace_id}")
        return workspace_id, "created"
    else:
        raise RuntimeError(f"Workspace '{workspace_name}' could not be created or found.")

# -------------------------------
# Item Utilities
# -------------------------------

def fab_get_id(workspace_name, name):
    """
    Retrieves the item ID from a workspace.
    """
    return run_fab_command(f"get /{workspace_name}.Workspace/{name} -q id", capture_output=True, silently_continue=True)

def fab_get_display_name(workspace_name, name):
    """
    Retrieves the display name of an item.
    """
    return run_fab_command(f"get /{workspace_name}.Workspace/{name} -q displayName", capture_output=True, silently_continue=True)

def fab_get_items(workspace_id, item_id=''):
    """
    Retrieves item definitions or lists from a workspace.
    """
    if item_id:
        return run_fab_command(f"api -X post workspaces/{workspace_id}/items/{item_id}/getDefinition", capture_output=True, silently_continue=True)
    return run_fab_command(f"api -X get workspaces/{workspace_id}/items/{item_id}", capture_output=True, silently_continue=True)

# -------------------------------
# File and ID Replacement
# -------------------------------

def copy_to_tmp(name, child=None):
    """
    Extracts item files from a ZIP archive to a temporary directory.
    """
    child_path = "" if child is None else f".children/{child}/"
    shutil.rmtree("./builtin/tmp", ignore_errors=True)
    path2zip = "./builtin/src/src.zip"
    with ZipFile(path2zip) as archive:
        for file in archive.namelist():
            if file.startswith(f'src/{name}/{child_path}'):
                archive.extract(file, './builtin/tmp')
    return f"./builtin/tmp/src/{name}/{child_path}"

def replace_ids_in_folder(folder_path, mapping_table, environment_name):
    """
    Replaces old IDs with new ones in specified file types within a folder.
    """
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb', '.tmdl')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    for mapping in mapping_table:
                        if mapping["environment"] in (environment_name, "config"):
                            content = content.replace(mapping["old_id"], mapping["new_id"])
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)

def replace_ids_and_mark_inactive(folder_path, mapping_table, environment_name, target_guids):
    """
    Replaces old IDs with new ones in JSON-based files and deactivates activities
    that reference connections not in the target_guids list.

    Parameters:
    - folder_path (str): Path to the folder containing files to process.
    - mapping_table (list): List of dictionaries with 'old_id', 'new_id', and 'environment'.
    - environment_name (str): Current environment name to filter applicable mappings.
    - target_guids (list): List of valid connection GUIDs to retain as active.

    Returns:
    - None. Files are modified in-place.
    """
    import os
    import json

    def find_externalReferences_in_dict(j):
        externalReferences = {}
        for key, value in j.items():
            if isinstance(value, dict):
                externalReferences.update(find_externalReferences_in_dict(value))
            if key == "externalReferences":
                externalReferences[key] = value
        return externalReferences

    def should_deactivate(connection):
        return (
            connection not in target_guids and
            connection not in ['@item().ConnectionGuid', '@pipeline().parameters.ConnectionGuid']
        )

    def process_nested_activities(activities):
        for activity in activities:
            result = find_externalReferences_in_dict(activity)
            connection = result.get('externalReferences', {}).get('connection')
            if connection and should_deactivate(connection):
                print(f"Deactivate activity {activity.get('name')} for connection {connection}")
                activity["state"] = "Inactive"
                activity["onInactiveMarkAs"] = "Succeeded"

    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb', '.tmdl')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)

                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()

                # Replace IDs
                for mapping in mapping_table:
                    if mapping["environment"] in (environment_name, "config"):
                        if  mapping["new_id"] != None:
                            content = content.replace(mapping["old_id"], mapping["new_id"])

                try:
                    data = json.loads(content)
                except json.JSONDecodeError:
                    continue

                if not data or not data.get("properties") or not data["properties"].get("activities"):
                    continue

                for activity in data["properties"]["activities"]:
                    process_nested_activities([activity])
                    for key in ["activities", "ifFalseActivities", "ifTrueActivities"]:
                        nested = activity.get("typeProperties", {}).get(key)
                        if nested:
                            process_nested_activities(nested)

                content = json.dumps(data, indent=2)
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)


# -------------------------------
# Description and Identity Assignment
# -------------------------------

def assign_workspace_description(workspace):
    """
    Assigns a standard description to the workspace.
    """
    workspace_name = workspace['name']
    payload = 'This items are generated by the FMD Framework, everytime you run the setup Notebook all changes are overwritten. Check for more details on https://github.com/edkreuk/FMD_FRAMEWORK'
    run_fab_command(f'set "/{workspace_name}.workspace -q description -i {payload} -f', silently_continue=True)
    print(f" - Description applied to '{workspace_name}'")

def assign_item_description(workspace, item):
    """
    Assigns a standard description to an item.
    """
    workspace_name = workspace['name']
    payload = 'This Item is initially generated by the FMD Framework, check for more details on https://github.com/edkreuk/FMD_FRAMEWORK'
    run_fab_command(f'set "/{workspace_name}.workspace/{item} -q description -i {payload} -f', silently_continue=True)
    print(f" - Description applied to {item} in '{workspace_name}'")

def assign_managed_identity(workspace):
    """
    Assigns a managed identity to the workspace.
    """
    workspace_name = workspace['name']
    try:
        run_fab_command(f'create "/{workspace_name}.workspace/.managedidentities/{workspace_name}.ManagedIdentity"', silently_continue=True)
        print(f" - Managed identity assigned to '{workspace_name}'")
    except Exception as e:
        print(f" - Failed to assign managed identity: {e}")

# -------------------------------
# Role Assignment
# -------------------------------

def assign_roles(workspace):
    """
    Assigns roles to principals in the workspace.
    """
    workspace_path = f"/{workspace['name']}.workspace"
    print(f" - Assigning Workspace roles")
    for role in workspace['roles']:
        try:
            print(f"Assigning role '{role['role']}' to '{role['principal']['displayName']}' in workspace '{workspace['name']}'")
            run_fab_command(
                f'acl set "{workspace_path}" -I {role["principal"]["id"]} -R {role["role"]} -f',
                silently_continue=True
            )
        except Exception as e:
            print(f" - Failed to assign role: {e}")


# -------------------------------
# Folder Handling
# -------------------------------

def fab_get_folders(workspace_id):
    """
    Retrieves all folders in a workspace.
    """
    response = run_fab_command(f"api workspaces/{workspace_id}/folders", capture_output=True, silently_continue=True)
    return json.loads(response).get('text', {}).get('value', [])

def fab_get_folder(workspace_id, folder_name):
    """
    Retrieves folder metadata by name.
    """
    for f in fab_get_folders(workspace_id):
        if f.get('displayName') == folder_name:
            return f
    return None

def fab_assign_item_folder(workspace_id, item_id, folder):
    """
    Assigns an item to a folder, creating the folder if it doesn't exist.
    """
    folder_details = fab_get_folder(workspace_id, folder)
    if folder_details is None:
        payload = json.dumps({"displayName": folder})
        print(f"Folder does not exist, creating {payload}")
        folder_details = run_fab_command(f"api -X post workspaces/{workspace_id}/folders -i {payload}", capture_output=True, silently_continue=False)
        folder_details = json.loads(folder_details).get('text', {})
    payload = json.dumps({'folder': folder_details.get('id')})
    result = run_fab_command(f"api -X patch workspaces/{workspace_id}/items/{item_id} -i {payload}", capture_output=True, silently_continue=False)
    print(result)


def deploy_workspaces(workspace, environment_name, old_id, mapping_table, tasks):
    """
    Deploys a workspace by ensuring its existence, assigning identity, roles, and description.
    Updates the mapping table and logs the deployment task.

    Parameters:
    - workspace (dict): Workspace configuration including name and capacity.
    - environment_name (str): Target environment name.
    - old_id (str): Previous workspace ID to be replaced.
    - mapping_table (list): List to store ID mappings.
    - tasks (list): List to store task execution logs.
    """
    start = time()
    print("\n#############################################")
    print(f" - Processing: workspace {workspace['name']}")

    workspace_id, status = ensure_workspace_exists(workspace)
    workspace["id"] = workspace_id

    print("--------------------------")
    print(f"Updating Mapping Table: {environment_name}")
    mapping_table.append({
        "Description": workspace['name'],
        "environment": environment_name,
        "old_id": old_id,
        "new_id": workspace_id
    })
    mapping_table.append({
        "Description": workspace['name'],
        "environment": environment_name,
        "old_id": "00000000-0000-0000-0000-000000000000",
        "new_id": workspace_id
    })

    assign_managed_identity(workspace)
    assign_roles(workspace)
    assign_workspace_description(workspace)

    tasks.append({
        "task_name": f"Update workspace {workspace['name']}",
        "task_duration": int(time() - start),
        "status": "success"
    })



def deploy_item(workspace, name, mapping_table, environment_name, connection_list, tasks, lakehouse_schema_enabled, child=None, it=None):
    """
    Deploys an item (Notebook, Lakehouse, DataPipeline) into a workspace.
    Handles ID replacement, description assignment, and updates mapping and task logs.

    Parameters:
    - workspace (dict): Workspace configuration including name.
    - name (str): Name of the item to deploy.
    - mapping_table (list): List to store ID mappings.
    - environment_name (str): Target environment name.
    - connection_list (list): List of valid connection GUIDs.
    - tasks (list): List to store task execution logs.
    - lakehouse_schema_enabled (bool): Flag to enable schema creation for lakehouses.
    - child (str, optional): Child item name if applicable.
    - it (dict, optional): Item metadata including old ID.
    """
    start = time()
    print("\n#############################################")
    print(f"Deploying in {workspace['name']}: {name}")

    tmp_path = copy_to_tmp(name, child)
    name = name if child is None else child
    workspace_name = workspace['name']
    workspace_id = get_workspace_id_by_name(workspace_name)
    cli_parameter = ''

    if "Notebook" in name:
        cli_parameter += " --format .py"
        result = run_fab_command(
            f"import / {workspace_name}.Workspace/{name} -i {tmp_path} -f {cli_parameter}",
            capture_output=True, silently_continue=True, raw_output=False
        )
        assign_item_description(workspace, name)
        new_id = fab_get_id(workspace_name, name)

    elif "Lakehouse" in name:
        if lakehouse_schema_enabled:
            result = run_fab_command(
                f"create {workspace_name}.Workspace/{name} -P enableschemas=true",
                capture_output=True, silently_continue=True, raw_output=False
            )
        else:
            result = run_fab_command(
                f"create {workspace_name}.Workspace/{name} -P enableschemas=false",
                capture_output=True, silently_continue=True, raw_output=False
            )
        assign_item_description(workspace, name)
        new_id = fab_get_id(workspace_name, name)

    elif "DataPipeline" in name:
        print(f"Replacing connections guid in {workspace['name']}: {name}")
        replace_ids_and_mark_inactive(tmp_path, mapping_table, environment_name, connection_list)
        result = run_fab_command(
            f"import / {workspace_name}.Workspace/{name} -i {tmp_path} -f {cli_parameter}",
            capture_output=True, silently_continue=True, raw_output=False
        )
        assign_item_description(workspace, name)
        new_id = fab_get_id(workspace_name, name)

    elif "VariableLibrary" in name:   #Not working yet, import is giving error back
        print(f"Creating {workspace['name']}: {name}")
        result = run_fab_command(
            f"import / {workspace_name}.Workspace/{name} -i {tmp_path} -f",
            capture_output=True, silently_continue=True, raw_output=False
        )

        new_id = fab_get_id(workspace_name, name)
    print(result)
    if it:
        mapping_table.append({
            "Description": name,
            "environment": environment_name,
            "old_id": it["id"],
            "new_id": new_id
        })

    tasks.append({
        "task_name": f"Update item Definition {workspace_name} - {name}",
        "task_duration": int(time() - start),
        "status": result
    })



## Load configuration
Create  workspace, identity and roles

In [ ]:
base_path = './builtin/'
config_path = os.path.join(base_path, 'config/item_config.yaml')

with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

deploy_order_path = os.path.join(base_path, 'config/item_deployment.json')
with open(deploy_order_path, 'r') as file:
        item_deployment =json. load(file)

deploy_order_path = os.path.join(base_path, 'config/sql_deployment.json')
with open(deploy_order_path, 'r') as file:
        sql_deployment =json. load(file)

deploy_order_path = os.path.join(base_path, 'config/data_deployment.json')
with open(deploy_order_path, 'r') as file:
        data_deployment =json. load(file)

deploy_order_path = os.path.join(base_path, 'config/lakehouse_deployment.json')
with open(deploy_order_path, 'r') as file:
        lakehouse_deployment =json. load(file)

mapping_table=[]

### Deploy workspaces(Code and Data)

In [ ]:
for environment in environments:
    print(f"--------------------------")
    print(f"Updating Workspace: {environment['environment_name']}")
    deploy_workspaces(workspace=environment['workspaces']['code'], environment_name=environment['environment_name'], old_id=config["workspaces"]["workspace_code"], mapping_table=mapping_table, tasks=tasks)
    deploy_workspaces(workspace=environment['workspaces']['data'], environment_name=environment['environment_name'], old_id=config["workspaces"]["workspace_data"], mapping_table=mapping_table, tasks=tasks)
    
    # Append the remaining connections
    mapping_table.append({"Description": "CON_FMD_FABRIC_SQL" ,"environment": environment['environment_name'],"old_id": config['connections']["CON_FMD_FABRIC_SQL"], "new_id": environment['connections']['CON_FMD_FABRIC_SQL']})
    mapping_table.append({"Description": "CON_FMD_FABRIC_PIPELINES" ,"environment": environment['environment_name'] ,"old_id": config["connections"]["CON_FMD_FABRIC_PIPELINES"], "new_id": environment['connections']['CON_FMD_FABRIC_PIPELINES']})
    mapping_table.append({"Description": "CON_FMD_ADF_PIPELINES" ,"environment": environment['environment_name'] ,"old_id": config["connections"]["CON_FMD_ADF_PIPELINES"], "new_id": environment['connections']['CON_FMD_ADF_PIPELINES']})

print(f"--------------------------")   


## Deploy Workspace(Config)

In [ ]:
print(f"Updating Workspace: Configuration")
deploy_workspaces(workspace=configuration['workspace'], environment_name='config', old_id=config["workspaces"]["workspace_config"], mapping_table=mapping_table, tasks=tasks)


## Deploy Lakehouses

In [ ]:
for environment in environments:
    print(f"\n--------------------------")
    print(f"Processing: {environment['environment_name']}")
    for workspace in [environment['workspaces']['data']]:
        exclude = []

        for it in lakehouse_deployment:

            new_id = None
            
            name = it["name"]
            type = it["type"]

            if name in exclude:
                continue
            deploy_item(workspace,name,mapping_table, environment['environment_name'],connection_list, tasks, lakehouse_schema_enabled,None,it)
        

## Deploy Fabric Database

In [ ]:
for target_item in data_deployment:
    if target_item['type'] in ('SQLDatabase','SQLEndpoint'):
        target_item['name'] = configuration['DatabaseName']

start = time()
print(f"\n -----")
print(f" - Processing: workspace {configuration['workspace']['name']}")
workspace_id, status = ensure_workspace_exists(configuration['workspace'])
empty = True

   
target_items = fab_get_items(workspace_id)
target_items = json.loads(target_items)["text"]

for deployment_item in data_deployment:
    for target_item in target_items['value']:

        if target_item['displayName'] == deployment_item['name'] \
                and target_item['type'] == deployment_item['type']:
            print(f" - Skip existing: {deployment_item['name']}, {deployment_item['type']}, {target_item['id']}")
            break
    else:
        
        print(f" - Creating: {deployment_item['name']} {deployment_item['type']} in workspace:{workspace_id} ")
        item = deployment_item.copy()
        
        if empty:
            if item.get('definition'):
                print(f" - Dropping definition")
                item.pop('definition')

        # Construct the JSON payload
        payload = json.dumps({"displayName": deployment_item['name'], "Description": "Fabric SQL Database used by FMD_FRAMEWORK"})
        if deployment_item['type'] in ('SQLDatabase'):
            try:
                raw_response = run_fab_command(
                    f"api -X post workspaces/{workspace_id}/SQLDatabases -i '{payload}'",
                    capture_output=True,
                    silently_continue=True
                )
                response = json.loads(raw_response)

                if isinstance(response, dict) and response.get("status_code", 200) >= 400:
                    print(f"{response.get('status_code')=}\n{response.get('reason')=}\n{response.get('text')=}\n{url=}\n{payload=}\n{payloadtype=}")
                    if response.get("errorCode"):
                        print(f"{response['errorCode']}")
            except Exception as e:
                print(f"Error: {e}. Retrying in 60 seconds...")
                sleep(60)
                raw_response = run_fab_command(
                    f"api -X post workspaces/{workspace_id}/SQLDatabases -i '{payload}'",
                    capture_output=True,
                    silently_continue=True
                )
                response = json.loads(raw_response)

            #wait so SQL Database can be deployed and we can pick up the Guid after
            sleep(60)

## Get Fabric database configuration

In [ ]:
for target_item in data_deployment:
    if target_item['type'] in ('SQLDatabase','SQLEndpoint'):
        target_item['name'] = configuration['DatabaseName']
        
start = time()
print(f"\n -----")
print(f" - Processing: workspace {configuration['workspace']['name']}")
workspace_id, status = ensure_workspace_exists(configuration['workspace'])
empty = True

   
target_items = fab_get_items(workspace_id)
target_items = json.loads(target_items)["text"]
for deployment_item in data_deployment:
    for target_item in target_items['value']:

        print(f" - Updating mapping table: {deployment_item['name']} {deployment_item['type']} ")
  
        if deployment_item['type'] in ('SQLDatabase'):
            if deployment_item.get('endpoint', '') != '':
                return_item =  run_fab_command(f"api -X get workspaces/{workspace_id}/SQLDatabases/{target_item['id']}" , capture_output = True, silently_continue= True)
                return_item = json.loads(return_item)["text"]
                if deployment_item['type'] in ('SQLDatabase'):
                    if return_item.get("properties", {}).get("serverFqdn", "") != '':
                        deployment_item["connectionString"] = return_item["properties"]["serverFqdn"].replace(',1433', '')
                        mapping_table.append({"Description":deployment_item['name'] , "environment": 'config', "old_id": config["database"]["id"], "new_id": target_item['id']})
                        mapping_table.append({"Description":deployment_item['name'] , "environment": 'config', "old_id": deployment_item["endpoint"], "new_id": deployment_item["connectionString"]})
                        tasks.append({"task_name":f"Update item  Definition  {configuration['workspace']['name']} - {name}", "task_duration": int(time() - start), "status": "success"})
                    if return_item.get("properties", {}).get("databaseName", "") != '':
                        deployment_item["databaseName"] = return_item["properties"]["databaseName"]


## Deploy Items

In [ ]:
for environment in environments:
    print(f"\n--------------------------")
    print(f"Processing: {environment['environment_name']}")
    for workspace in [environment['workspaces']['code']]:
        exclude = []

        for it in item_deployment:

            new_id = None
            
            name = it["name"]
            type = it["type"]

            if name in exclude:
                continue
            deploy_item(workspace,name,mapping_table,environment['environment_name'],connection_list, tasks, lakehouse_schema_enabled, None,it)
            

            for child in it.get("children",[]):
                child_name = child["name"]
                print(child_name)
                deploy_item(workspace,name,mapping_table, environment['environment_name'],connection_list, tasks, lakehouse_schema_enabled,child_name,child) 

### Create SQL deployment Manifest

### Add Connection to Fabric Database

In [ ]:
custom_sql_deployment = {"queries_stored_procedures": []}
for connection in connections['value']:
    
    display_name = connection.get('displayName', '')
    if display_name and display_name.startswith('CON_FMD'):
        connection_type = connection.get('connectionDetails', {}).get('type', 'Unknown')
        connection_id = connection.get('id')
      
        exec_statement = (
            f"EXEC [integration].[sp_UpsertConnection] "
            f"@ConnectionGuid = \"{connection_id}\", "
            f"@Name = \"{display_name}\", "
            f"@Type = \"{connection_type}\", "
            f"@IsActive = 1"
        )
        custom_sql_deployment["queries_stored_procedures"].append(exec_statement)


### Add Workspaces to Fabric Database

In [ ]:
workspaces = []
workspaces.append(configuration['workspace'])

for environment in environments:
    workspaces.append(environment['workspaces']['code'])
    workspaces.append(environment['workspaces']['data'])
    
for workspace in workspaces:
    print(f'EXEC [integration].[sp_UpsertWorkspace](@WorkspaceId = "{workspace["id"]}" ,@Name = "{workspace["name"]}")')
    custom_sql_deployment["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertWorkspace] @WorkspaceId = "{workspace["id"]}", @Name = "{workspace["name"]}"')

### Add Data Pipelines to Fabric Database

In [ ]:
for environment in environments:
    result = run_fab_command(f"api -X get workspaces/{environment['workspaces']['code']['id']}/items", capture_output=True, silently_continue=True)
    existing_items = json.loads(result)['text']
    for item in existing_items.get('value', []):
        if item['type'] == 'DataPipeline':
            print(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["code"]["id"]}" ,@Name = "{item["displayName"]}"')
            custom_sql_deployment["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertPipeline] @PipelineId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

### Add Lakehouses to Fabric Database

In [ ]:
for environment in environments:
    result = run_fab_command(f"api -X get workspaces/{environment['workspaces']['data']['id']}/items", capture_output=True, silently_continue=True)
    existing_items = json.loads(result)['text']
    for item in existing_items.get('value', []):
        if item['type'] == 'Lakehouse':
            print(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')
            custom_sql_deployment["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertLakehouse] @LakehouseId = "{item["id"]}", @WorkspaceId = "{environment["workspaces"]["data"]["id"]}" ,@Name = "{item["displayName"]}"')

### Add Demo data for testing to Fabric Database

In [ ]:
if load_demo_data_data:  
    demo_sql_deployment = {"queries_stored_procedures": []}
    demo_sql_deployment["queries_stored_procedures"].append(f'EXEC [integration].[sp_UpsertConnection] @ConnectionGuid = "00000000-0000-0000-0000-000000000000", @Name = "CON_FMD_ONELAKE", @Type = "ONELAKE", @IsActive = 1')
    demo_sql_deployment["queries_stored_procedures"].append("""
        DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type='ONELAKE_TABLES_01')
        DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
        EXECUTE [integration].[sp_UpsertDataSource] 
            @ConnectionId = @ConnectionIdInternal
            ,@DataSourceId = @DataSourceIdInternal
            ,@Name = 'LH_DATA_LANDINGZONE'
            ,@Namespace = 'ONELAKE'
            ,@Type = 'ONELAKE_TABLES_01'
            ,@Description = 'ONELAKE_TABLES'
            ,@IsActive = 1
    """)
    demo_sql_deployment["queries_stored_procedures"].append("""
        DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type ='ONELAKE_FILES_01')
        DECLARE @ConnectionIdInternal INT = (SELECT ConnectionId FROM integration.Connection WHERE ConnectionGuid = '00000000-0000-0000-0000-000000000000')
        EXECUTE [integration].[sp_UpsertDataSource] 
            @ConnectionId = @ConnectionIdInternal
            ,@DataSourceId = @DataSourceIdInternal
            ,@Name = 'LH_DATA_LANDINGZONE'
            ,@Namespace = 'ONELAKE'
            ,@Type = 'ONELAKE_FILES_01'
            ,@Description = 'ONELAKE_FILES'
            ,@IsActive = 1
    """)
    demo_sql_deployment["queries_stored_procedures"].append("""
        DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
        DECLARE @DataSourceIdInternal INT = (SELECT DataSourceId FROM integration.DataSource WHERE Name = 'LH_DATA_LANDINGZONE' and Type='ONELAKE_TABLES_01')
        DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_DATA_LANDINGZONE')
        EXECUTE [integration].[sp_UpsertLandingzoneEntity] 
            @LandingzoneEntityId = @LandingzoneEntityIdInternal
            ,@DataSourceId = @DataSourceIdInternal
            ,@LakehouseId = @LakehouseIdInternal
            ,@SourceSchema = 'in'
            ,@SourceName = 'customer'
            ,@SourceCustomSelect = ''
            ,@FileName = 'customer'
            ,@FilePath = 'fmd'
            ,@FileType = 'parquet'
            ,@IsIncremental = 0
            ,@IsIncrementalColumn = ''
            ,@IsActive = 1
    """)
    demo_sql_deployment["queries_stored_procedures"].append("""
        DECLARE @LandingzoneEntityIdInternal INT = (SELECT LandingzoneEntityId FROM integration.LandingzoneEntity WHERE SourceSchema = 'in' and SourceName = 'customer')
        DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
        DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_BRONZE_LAYER')
        EXECUTE [integration].[sp_UpsertBronzeLayerEntity] 
            @BronzeLayerEntityId = @BronzeLayerEntityIdInternal
            ,@LandingzoneEntityId = @LandingzoneEntityIdInternal
            ,@Schema = 'in'
            ,@Name = 'customer'
            ,@FileType = 'Delta'
            ,@LakehouseId = @LakehouseIdInternal
            ,@PrimaryKeys = 'CustomerId'
            ,@IsActive = 1
    """)
    demo_sql_deployment["queries_stored_procedures"].append("""
        DECLARE @BronzeLayerEntityIdInternal INT = (SELECT BronzeLayerEntityId FROM integration.BronzeLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
        DECLARE @SilverLayerEntityIdInternal INT = (SELECT SilverLayerEntityId FROM integration.SilverLayerEntity WHERE [Schema] = 'in' and [Name] = 'customer')
        DECLARE @LakehouseIdInternal INT = (SELECT top 1 LakehouseId FROM integration.Lakehouse WHERE Name = 'LH_SILVER_LAYER')
        EXECUTE [integration].[sp_UpsertSilverLayerEntity] 
            @SilverLayerEntityId = @SilverLayerEntityIdInternal
            ,@BronzeLayerEntityId = @BronzeLayerEntityIdInternal
            ,@LakehouseId = @LakehouseIdInternal
            ,@Name = 'customer'
            ,@Schema = 'in'
            ,@FileType = 'delta'
            ,@IsActive = 1
    """)

## Deploy SQL Code

In [ ]:
for target_item in data_deployment:
    if target_item['type'] == 'SQLDatabase':
        connstring = target_item["connectionString"]
        database = target_item['databaseName']


try:
    i = 0

    token = notebookutils.credentials.getToken('pbi').encode('utf-16-le')
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)

    print(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};")
    connection = pyodbc.connect(f"DRIVER={driver};SERVER={connstring};PORT=1433;DATABASE={database};", attrs_before={1256:token_struct}, timeout=12)

    with connection.cursor() as cursor:
        cursor.execute("SELECT 1")  # Execute the warm-up query (a simple query like 'SELECT 1' can be used)
        cursor.fetchone()
        connection.timeout = 10  # Setting a lower timeout for subsequent queries
    for it in sql_deployment:
        for i, query in enumerate(it["queries_tables"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(it["queries_views"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(it["queries_stored_procedures"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
        for i, query in enumerate(it["queries_logging"]):
            print(f' - execute "{query}"')
            cursor.execute(query)
            cursor.commit()
    for i, query in enumerate(custom_sql_deployment["queries_stored_procedures"]):
        print(f' - execute "{query}"')
        cursor.execute(query)
        cursor.commit()
    for i, query in enumerate(demo_sql_deployment["queries_stored_procedures"]):
        print(f' - execute "{query}"')
        cursor.execute(query)
        cursor.commit()

    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"success"})
except pyodbc.OperationalError as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"pyodbc failed: {e}"})
except Exception as e:
    print(e) 
    tasks.append({"task_name":f"{workspace.get('displayName')} {database} query {i}", "task_duration": 1, "status": f"failed: {e}"})

In [ ]:
display(tasks)